In [ ]:
# Imports
import glob
import os
from PIL import Image
import warnings

# For a progress bar
import tqdm

import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bebi103

import altair as alt

import altair_catplot as altcat

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

bokeh.io.output_notebook()

In [ ]:
def imshow(im, color_mapper=None, plot_height=400, plot_width=None,
           length_units='pixels', interpixel_distance=1.0,
           x_range=None, y_range=None, colorbar=False,
           no_ticks=False, x_axis_label=None, y_axis_label=None,
           title=None, flip=True, return_im=False,
           saturate_channels=True, min_intensity=None,
           max_intensity=None, display_clicks=False, record_clicks=False):
    """
    Display an image in a Bokeh figure. Written by Professor Justin Bois, Caltech
    Parameters
    ----------
    im : Numpy array
        If 2D, intensity image to be displayed. If 3D, first two
        dimensions are pixel values. Last dimension can be of length
        1, 2, or 3, which specify colors.
    color_mapper : str or bokeh.models.LinearColorMapper, default None
        If `im` is an intensity image, `color_mapper` is a mapping of
        intensity to color. If None, default is 256-level Viridis.
        If `im` is a color image, then `color_mapper` can either be
        'rgb' or 'cmy' (default), for RGB or CMY merge of channels.
    plot_height : int
        Height of the plot in pixels. The width is scaled so that the
        x and y distance between pixels is the same.
    plot_width : int or None (default)
        If None, the width is scaled so that the x and y distance
        between pixels is approximately the same. Otherwise, the width
        of the plot in pixels.
    length_units : str, default 'pixels'
        The units of length in the image.
    interpixel_distance : float, default 1.0
        Interpixel distance in units of `length_units`.
    x_range : bokeh.models.Range1d instance, default None
        Range of x-axis. If None, determined automatically.
    y_range : bokeh.models.Range1d instance, default None
        Range of y-axis. If None, determined automatically.
    colorbar : bool, default False
        If True, include a colorbar.
    no_ticks : bool, default False
        If True, no ticks are displayed. See note below.
    x_axis_label : str, default None
        Label for the x-axis. If None, labeled with `length_units`.
    y_axis_label : str, default None
        Label for the y-axis. If None, labeled with `length_units`.
    title : str, default None
        The title of the plot.
    flip : bool, default True
        If True, flip image so it displays right-side up. This is
        necessary because traditionally images have their 0,0 pixel
        index in the top left corner, and not the bottom left corner.
    return_im : bool, default False
        If True, return the GlyphRenderer instance of the image being
        displayed.
    saturate_channels : bool, default True
        If True, each of the channels have their displayed pixel values
        extended to range from 0 to 255 to show the full dynamic range.
    min_intensity : int or float, default None
        Minimum possible intensity of a pixel in the image. If None,
        the image is scaled based on the dynamic range in the image.
    max_intensity : int or float, default None
        Maximum possible intensity of a pixel in the image. If None,
        the image is scaled based on the dynamic range in the image.
    display_clicks : bool, default False
        If True, display clicks to the right of the plot using
        JavaScript. The clicks are not recorded nor stored, just
        printed. If you want to store the clicks, use the
        `record_clicks()` or `draw_rois()` functions.
    record_clicks : bool, default False
        Deprecated. Use `display_clicks`.
    Returns
    -------
    p : bokeh.plotting.figure instance
        Bokeh plot with image displayed.
    im : bokeh.models.renderers.GlyphRenderer instance (optional)
        The GlyphRenderer instance of the image being displayed. This is
        only returned if `return_im` is True.
    Notes
    -----
    .. The plot area is set to closely approximate square pixels, but
       this is not always possible since Bokeh sets the plotting area
       based on the entire plot, inclusive of ticks and titles. However,
       if you choose `no_ticks` to be True, no tick or axes labels are
       present, and the pixels are displayed as square.
    """
    if record_clicks:
        warnings.warn(
            '`record_clicks` is deprecated. Use the `bebi103.viz.record_clicks()` function to store clicks. Otherwise use the `display_clicks` kwarg to print the clicks to the right of the displayed image.',
            DeprecationWarning)

    # If a single channel in 3D image, flatten and check shape
    if im.ndim == 3:
        if im.shape[2] == 1:
            im = im[:,:,0]
        elif im.shape[2] not in [2, 3]:
            raise RuntimeError('Can only display 1, 2, or 3 channels.')

    # If binary image, make sure it's int
    if im.dtype == bool:
        im = im.astype(np.uint8)

    # Get color mapper
    if im.ndim == 2:
        if color_mapper is None:
            color_mapper = bokeh.models.LinearColorMapper(
                                        bokeh.palettes.viridis(256))
        elif (type(color_mapper) == str
                and color_mapper.lower() in ['rgb', 'cmy']):
            raise RuntimeError(
                    'Cannot use rgb or cmy colormap for intensity image.')
        if min_intensity is None:
            color_mapper.low = im.min()
        else:
            color_mapper.low = min_intensity
        if max_intensity is None:
            color_mapper.high = im.max()
        else:
            color_mapper.high = max_intensity
    elif im.ndim == 3:
        if color_mapper is None or color_mapper.lower() == 'cmy':
            im = im_merge(*np.rollaxis(im, 2),
                          cmy=True,
                          im_0_min=min_intensity,
                          im_1_min=min_intensity,
                          im_2_min=min_intensity,
                          im_0_max=max_intensity,
                          im_1_max=max_intensity,
                          im_2_max=max_intensity)
        elif color_mapper.lower() == 'rgb':
            im = im_merge(*np.rollaxis(im, 2),
                          cmy=False,
                          im_0_min=min_intensity,
                          im_1_min=min_intensity,
                          im_2_min=min_intensity,
                          im_0_max=max_intensity,
                          im_1_max=max_intensity,
                          im_2_max=max_intensity)
        else:
            raise RuntimeError('Invalid color mapper for color image.')
    else:
        raise RuntimeError(
                    'Input image array must have either 2 or 3 dimensions.')

    # Get shape, dimensions
    n, m = im.shape[:2]
    if x_range is not None and y_range is not None:
        dw = x_range[1] - x_range[0]
        dh = y_range[1] - y_range[0]
    else:
        dw = m * interpixel_distance
        dh = n * interpixel_distance
        x_range = [0, dw]
        y_range = [0, dh]

    # Set up figure with appropriate dimensions
    if plot_width is None:
        plot_width = int(m/n * plot_height)
    if colorbar:
        plot_width += 40
        toolbar_location = 'above'
    else:
        toolbar_location = 'right'
    p = bokeh.plotting.figure(plot_height=plot_height,
                              plot_width=plot_width,
                              x_range=x_range,
                              y_range=y_range,
                              title=title,
                              toolbar_location=toolbar_location,
                              tools='pan,box_zoom,wheel_zoom,reset,save')
    if no_ticks:
        p.xaxis.major_label_text_font_size = '0pt'
        p.yaxis.major_label_text_font_size = '0pt'
        p.xaxis.major_tick_line_color = None
        p.xaxis.minor_tick_line_color = None
        p.yaxis.major_tick_line_color = None
        p.yaxis.minor_tick_line_color = None
    else:
        if x_axis_label is None:
            p.xaxis.axis_label = length_units
        else:
            p.xaxis.axis_label = x_axis_label
        if y_axis_label is None:
            p.yaxis.axis_label = length_units
        else:
            p.yaxis.axis_label = y_axis_label

    # Display the image
    if im.ndim == 2:
        if flip:
            im = im[::-1,:]
        im_bokeh = p.image(image=[im],
                           x=x_range[0],
                           y=y_range[0],
                           dw=dw,
                           dh=dh,
                           color_mapper=color_mapper)
    else:
        im_bokeh = p.image_rgba(image=[rgb_to_rgba32(im, flip=flip)],
                                x=x_range[0],
                                y=y_range[0],
                                dw=dw,
                                dh=dh)

    # Make a colorbar
    if colorbar:
        if im.ndim == 3:
            warnings.warn('No colorbar display for RGB images.')
        else:
            color_bar = bokeh.models.ColorBar(color_mapper=color_mapper,
                                              label_standoff=12,
                                              border_line_color=None,
                                              location=(0,0))
            p.add_layout(color_bar, 'right')

    if record_clicks or display_clicks:
        div = bokeh.models.Div(width=200)
        layout = bokeh.layouts.row(p, div)
        p.js_on_event(bokeh.events.Tap,
                      _display_clicks(div, attributes=['x', 'y']))
        if return_im:
            return layout, im_bokeh
        else:
            return layout

    if return_im:
        return p, im_bokeh
    return p

In [ ]:
def im_merge(im_0, im_1, im_2=None, im_0_max=None,
             im_1_max=None, im_2_max=None, im_0_min=None,
             im_1_min=None, im_2_min=None, cmy=True):
    """
    Merge channels to make RGB image.
    Parameters
    ----------
    im_0: array_like
        Image represented in first channel.  Must be same shape
        as `im_1` and `im_2` (if not None).
    im_1: array_like
        Image represented in second channel.  Must be same shape
        as `im_1` and `im_2` (if not None).
    im_2: array_like, default None
        Image represented in third channel.  If not None, must be same
        shape as `im_0` and `im_1`.
    im_0_max : float, default max of inputed first channel
        Maximum value to use when scaling the first channel. If None,
        scaled to span entire range.
    im_1_max : float, default max of inputed second channel
        Maximum value to use when scaling the second channel
    im_2_max : float, default max of inputed third channel
        Maximum value to use when scaling the third channel
    im_0_min : float, default min of inputed first channel
        Maximum value to use when scaling the first channel
    im_1_min : float, default min of inputed second channel
        Minimum value to use when scaling the second channel
    im_2_min : float, default min of inputed third channel
        Minimum value to use when scaling the third channel
    cmy : bool, default True
        If True, first channel is cyan, second is magenta, and third is
        yellow. Otherwise, first channel is red, second is green, and
        third is blue.
    Returns
    -------
    output : array_like, dtype float, shape (*im_0.shape, 3)
        RGB image.
    """

    # Compute max intensities if needed
    if im_0_max is None:
        im_0_max = im_0.max()
    if im_1_max is None:
        im_1_max = im_1.max()
    if im_2 is not None and im_2_max is None:
        im_2_max = im_2.max()

    # Compute min intensities if needed
    if im_0_min is None:
        im_0_min = im_0.min()
    if im_1_min is None:
        im_1_min = im_1.min()
    if im_2 is not None and im_2_min is None:
        im_2_min = im_2.min()

    # Make sure maxes are ok
    if im_0_max < im_0.max() or im_1_max < im_1.max() \
            or (im_2 is not None and im_2_max < im_2.max()):
        raise RuntimeError(
                'Inputted max of channel < max of inputted channel.')

    # Make sure mins are ok
    if im_0_min > im_0.min() or im_1_min > im_1.min() \
            or (im_2 is not None and im_2_min > im_2.min()):
        raise RuntimeError(
                'Inputted min of channel > min of inputted channel.')

    # Scale the images
    if im_0_max > im_0_min:
        im_0 = (im_0 - im_0_min) / (im_0_max - im_0_min)
    else:
        im_0 = (im_0 > 0).astype(float)

    if im_1_max > im_1_min:
        im_1 = (im_1 - im_1_min) / (im_1_max - im_1_min)
    else:
        im_0 = (im_0 > 0).astype(float)

    if im_2 is None:
        im_2 = np.zeros_like(im_0)
    elif im_2_max > im_2_min:
        im_2 = (im_2 - im_2_min) / (im_2_max - im_2_min)
    else:
        im_0 = (im_0 > 0).astype(float)

    # Convert images to RGB
    if cmy:
        im_c = np.stack((np.zeros_like(im_0), im_0, im_0), axis=2)
        im_m = np.stack((im_1, np.zeros_like(im_1), im_1), axis=2)
        im_y = np.stack((im_2, im_2, np.zeros_like(im_2)), axis=2)
        im_rgb = im_c + im_m + im_y
        for i in [0, 1, 2]:
            im_rgb[:,:,i] /= im_rgb[:,:,i].max()
    else:
        im_rgb = np.empty((*im_0.shape, 3))
        im_rgb[:,:,0] = im_0
        im_rgb[:,:,1] = im_1
        im_rgb[:,:,2] = im_2

    return im_rgb

In [ ]:
def rgb_to_rgba32(im, flip=True):
    """
    Convert an RGB image to a 32 bit-encoded RGBA image.
    Parameters
    ----------
    im : ndarray, shape (nrows, ncolums, 3)
        Input image. All pixel values must be between 0 and 1.
    flip : bool, default True
        If True, flip image so it displays right-side up. This is
        necessary because traditionally images have their 0,0 pixel
        index in the top left corner, and not the bottom left corner.
    Returns
    -------
    output : ndarray, shape (nros, ncolumns), dtype np.uint32
        Image decoded as a 32 bit RBGA image.
    """
    # Ensure it has three channels
    if im.ndim != 3 or im.shape[2] !=3:
        raise RuntimeError('Input image is not RGB.')

    # Make sure all entries between zero and one
    if (im < 0).any() or (im > 1).any():
        raise RuntimeError('All pixel values must be between 0 and 1.')

    # Get image shape
    n, m, _ = im.shape

    # Convert to 8-bit, which is expected for viewing
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        im_8 = skimage.img_as_ubyte(im)

    # Add the alpha channel, which is expected by Bokeh
    im_rgba = np.stack((*np.rollaxis(im_8, 2),
                        255*np.ones((n, m), dtype=np.uint8)), axis=2)

    # Reshape into 32 bit. Must flip up/down for proper orientation
    if flip:
        return np.flipud(im_rgba.view(dtype=np.int32).reshape((n, m)))
    else:
        return im_rgba.view(dtype=np.int32).reshape((n, m))


def rgb_frac_to_hex(rgb_frac):
    """
    Convert fractional RGB values to hexidecimal color string.
    Parameters
    ----------
    rgb_frac : array_like, shape (3,)
        Fractional RGB values; each entry is between 0 and 1.
    Returns
    -------
    str
        Hexidecimal string for the given RGB color.
    Examples
    --------
    >>> rgb_frac_to_hex((0.65, 0.23, 1.0))
    '#a53aff'
    >>> rgb_frac_to_hex((1.0, 1.0, 1.0))
    '#ffffff'
    """

    if len(rgb_frac) != 3:
        raise RuntimeError('`rgb_frac` must have exactly three entries.')

    if (np.array(rgb_frac) < 0).any() or (np.array(rgb_frac) > 1).any():
        raise RuntimeError('RGB values must be between 0 and 1.')

    return '#{0:02x}{1:02x}{2:02x}'.format(int(rgb_frac[0] * 255),
                                           int(rgb_frac[1] * 255),
                                           int(rgb_frac[2] * 255))


In [ ]:
def show_two_ims(im_1, 
                 im_2, 
                 titles=[None, None], 
                 interpixel_distances=[0.75488, 0.75488], # for 10x keyence. 20x keyence is 0.37744
                 length_units='µm',
                 color_mapper=None):
    
    """Convenient function for showing two images side by side."""
    
    p_1 = imshow(im_1,
                             plot_height=600,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units=length_units)
    p_2 = imshow(im_2,
                 plot_height=600,
                 title=titles[1],
                 color_mapper=color_mapper,
                 interpixel_distance=interpixel_distances[0],
                 length_units=length_units)
    
    p_1.xaxis.major_label_text_font_size = '12pt'
    p_1.yaxis.major_label_text_font_size = '12pt'

    p_2.xaxis.major_label_text_font_size = '12pt'
    p_2.yaxis.major_label_text_font_size = '12pt'

    p_1.xaxis.axis_label_text_font_size = '18pt'
    p_1.yaxis.axis_label_text_font_size = '18pt'

    p_2.xaxis.axis_label_text_font_size = '18pt'
    p_2.yaxis.axis_label_text_font_size = '18pt'
    
    
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

In [ ]:
def autofluorescence_subtract(autofluorescent_image, signal_image, box_width = 25):
    height = np.shape(signal_image)[0]
    width = np.shape(signal_image)[1]

    subtracted_image = np.empty(np.shape(signal_image))

    for i in range(0, box_width):
        for j in range(0, box_width):
            subtracted_image[i,j] = signal_image[i,j] - (autofluorescent_image[i,j]*np.mean(signal_image[i:i+box_width*2,j:j+box_width*2])/np.mean(autofluorescent_image[i:i+box_width*2,j:j+box_width*2]))

    for i in range(box_width, height - box_width):
        for j in range(box_width, width - box_width):
            subtracted_image[i,j] = signal_image[i,j] - (autofluorescent_image[i,j]*np.mean(signal_image[i - box_width:i + box_width, j-box_width:j+box_width])/np.mean(autofluorescent_image[i-box_width:i+box_width,j-box_width:j+box_width]))

    for i in range(height - box_width, height):
        for j in range(width - box_width, width):
            subtracted_image[i,j] = signal_image[i,j] - (autofluorescent_image[i,j]*np.mean(signal_image[i-box_width*2:i,j-box_width*2:j])/np.mean(autofluorescent_image[i-box_width*2:i,j-box_width*2:j]))
        
    return subtracted_image

In [ ]:
def plot_hist(hist_bin, title, y_axis_type='linear'):
    """Make plot of image histogram."""
    
    p = bokeh.plotting.figure(plot_height=300,
                              plot_width=400,
                              y_axis_type=y_axis_type,
                              x_axis_label='intensity',
                              y_axis_label='count',
                              title=title)
    hist, bins = hist_bin
    p.line(bins, hist, line_width=2)

    return p

In [ ]:
#template = im_pos[1440-1140:1440-1090, 1577:1627] > 0.05 #from Lung 67
#template = im_pos[1440-505:1440-493, 254:268] > 0.15 #From Lung 67
#template = im_pos[1440-1130:1440-1105, 1590:1615] > 0.05 #From Lung 67

#np.save('template.npy', template)

#bokeh.io.show(bebi103.viz.imshow(template, plot_height=900))

In [ ]:
template = np.load('template.npy')

bokeh.io.show(imshow(template, plot_height=900))

In [ ]:
# Load all of the images into a list
#fname = 'Lung_Paper_Images/Raw_Images/Kidney_RWT4_1-2-3-4/XY05/1_stitch_per_section_2_sections_XY05_00022_Overlay.tif'
#fname = 'Lung_Paper_Images/Test_Images/Liver/CAPA4_00034_Overlay.tif'
#fname = 'Lung_Paper_Images/Test_Images/Heart/v38_XY01_00024_Overlay.tif'
fname = 'Lung_Paper_Images/Test_Images/Brain/CAPA4_XY01_00063_Overlay.tif' #used for figure generation
#fname = 'Lung_Paper_Images/Test_Images/Brain/CAPA4/1_00069_Overlay.tif'

im_af = skimage.img_as_float(skimage.io.imread(fname)[:,:,0])
im_sig = skimage.img_as_float(skimage.io.imread(fname)[:,:,1])

bokeh.io.show(show_two_ims(im_af, im_sig, titles=['autofluorescence', 'signal'], interpixel_distances=[0.75488, 0.75488], color_mapper = None))

In [ ]:
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/autofluorescent.png', im_af)
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/signal.png', im_sig)
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/autofluorescent_mod.png', im_af*np.mean(im_sig)/np.mean(im_af))

In [ ]:
#im_sub = autofluorescence_subtract(im_af, im_sig, box_width = 8)
im_sub = im_sig - im_af*np.mean(im_sig)/np.mean(im_af)

im_pos = np.where(im_sub<0, 0, im_sub) 

bokeh.io.show(imshow(im_pos[206:506, 317:717],
                     plot_height=900,
                     interpixel_distance=0.75488, 
                     length_units='µm'))

#skimage.io.imsave('Group_Meetings/First_Year_Conversation_Images/autofluorescent.png', im_pos[206:506, 317:717])

In [ ]:
# Filter the image with a strong Gaussian blur
im_bg = skimage.filters.gaussian(im_pos, 5, truncate = 4)
#im_bg = skimage.restoration.denoise_bilateral(im_pos)

im_no_bg = im_pos - im_bg

im_no_bg = (im_no_bg - im_no_bg.min()) / (im_no_bg.max() - im_no_bg.min())

#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/Gaussian_Blur_AAV5.png', im_no_bg)
bokeh.io.show(imshow(im_no_bg[206:506, 317:717], plot_height=900))

In [ ]:
im_feat = skimage.feature.match_template(im_pos, template)

#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/Feature_Matching.png', im_feat)

bokeh.io.show(imshow(im_feat[200:500, 310:710], plot_height=900))

In [ ]:
#0.6 for Lung
#0.4 for Liver
#0.5 for Brain and Kidney

binary_thresh = im_feat > 0.4

binary_rem = skimage.morphology.remove_small_objects(binary_thresh, min_size=10)

bokeh.io.show(imshow(binary_rem[200:500, 310:710], plot_height=900))

In [ ]:
# Compute histograms
hist_bin = skimage.exposure.histogram(im_no_bg)
name = 'Subtracted Image'

hist, bins = hist_bin

dn_obj_dk = np.diff(np.diff(hist))

# p = bokeh.plotting.figure(plot_height=300,
#                           plot_width=500,
#                           x_axis_label='Intensity',
#                           y_axis_label='Second Derivative',
#                           background_fill_color='black',
#                           border_fill_color = 'black')

# p.xaxis.major_label_text_color = "white"
# p.yaxis.major_label_text_color = "white"

# p.xaxis.axis_label_text_color = "white"
# p.yaxis.axis_label_text_color = "white"


# p.line(bins, dn_obj_dk, line_width=2)
#bokeh.io.show(p)

In [ ]:
# 1000 for lung, 10000 for liver

past_i = 0
index = 0
thresh = []
for i in dn_obj_dk:
    if past_i > 5000 and (i) < 5000:
        thresh.append(bins[index-1] + (bins[index] - bins[index-1])*(5000 - past_i)/(i-past_i))
    past_i = i
    index += 1

In [ ]:
im_height = np.shape(im_pos)[0]
im_width = np.shape(im_pos)[1]
    
template_height = np.shape(template)[0]
template_width = np.shape(template)[1]

In [ ]:
max(thresh)

In [ ]:
binary_gauss = im_no_bg[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))] > max(thresh)
bokeh.io.show(imshow(binary_gauss[200:500, 310:710], plot_height = 1000))

In [ ]:
bokeh.io.show(bebi103.viz.imshow(binary_gauss*binary_rem, plot_height = 1000))

In [ ]:
im = binary_gauss
connectivity = 1
max_size = 1000

out = im.copy()

selem = ndi.generate_binary_structure(im.ndim, connectivity)
ccs = np.zeros_like(im, dtype=np.int32)
ndi.label(im, selem, output=ccs)

component_sizes = np.bincount(ccs.ravel())

too_big = component_sizes > max_size
too_big_mask = too_big[ccs]
out[too_big_mask] = 0

#bokeh.io.show(bebi103.viz.imshow(out, plot_height = 900))

In [ ]:
binary_mult = out*binary_rem

bokeh.io.show(imshow(binary_mult[200:500, 310:710], plot_height = 900))

In [ ]:
# Structuring element is radius 3 disk
binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=10)

bokeh.io.show(bebi103.viz.imshow(binary_rem, plot_height = 900))

In [ ]:
# Label binary image; backward kwarg says value in im_bw to consider backgr.
im_labeled, n_labels = skimage.measure.label(binary_rem, background=0, return_num=True)

# Show number of bacteria
print('Number of positive cells = ', n_labels)

# See result
# bokeh.io.show(
#     bebi103.viz.imshow(im_labeled,
#                        plot_height = 900,
#                        color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('rainbow')))

In [ ]:
# Load phase image
im_p = skimage.img_as_float(skimage.io.imread(fname))[:,:,:2]

# Build RGB image
im_p_rgb = np.dstack((im_p[:,:,0][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], im_p[:,:,1][int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))], binary_rem))

binary_rgb = np.dstack((np.zeros_like(binary_rem), binary_rem, np.zeros_like(binary_rem)))

# Show result
# bokeh.io.show(bebi103.viz.imshow(im_p_rgb,
#                                   color_mapper='rgb',
#                                   plot_height = 900))

In [ ]:
bokeh.io.show(show_two_ims(im_p[6:1435, 7:1914], binary_rgb, color_mapper='rgb'))

In [ ]:
bokeh.io.show(show_two_ims(im_p[206:506, 317:717], binary_rgb[200:500, 310:710], color_mapper='rgb'))

In [ ]:
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Pipeline/AAV5_Lung_original.png', skimage.img_as_float(skimage.io.imread(fname)))
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Brain/AAV9_Brain_original_zoom.png', skimage.img_as_float(skimage.io.imread(fname)[200:600, 600:1000]))
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Brain/AAV9_Brain_quant.png', im_p_rgb)
#skimage.io.imsave('One-on-one_Meetings/20190808_Meeting_Images/Brain/AAV9_Brain_quant_zoom.png', im_p_rgb[200:600, 600:1000])

In [ ]:
# See result
bokeh.io.show(
    bebi103.viz.imshow(im_labeled,
                       plot_height = 900,
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('rainbow')))

In [ ]:
cell_intensities = []

im_pos_crop = im_pos[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]

for count_value in range(1, np.max(im_labeled) + 1):
    inds = (im_labeled == count_value)
    cell_intensity = np.sum(im_pos_crop[inds])/np.sum(inds)
    cell_intensities.append(cell_intensity)

In [ ]:
p = bokeh.plotting.figure(y_axis_label = 'ECDF',
                          x_axis_label = 'Brightness per Cell')

p = bebi103.viz.ecdf(cell_intensities, 
                         plot_height=500, 
                         plot_width=1000,
                         line_width = 0.5,
                         p=p)
bokeh.io.show(p)